# 內容
- 學習搭建 Residual Block
- 學習搭建 Inception-ResNet Block

# 目的
- 了解 Residual Block 的原理
- 了解如何結合 Inception 與 Residual 概念

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, Input, BatchNormalization, Activation, BatchNormalization, Activation, Concatenate, Lambda
from keras import backend as K
from keras import layers

請參考 image/resnet_structure.png

# ResNet V1

In [ ]:
def residual_block_v1(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
   
    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

# 參考 ResNet V1 搭建 ResNet V2 版本的 Residual Block

In [ ]:
def residual_block_v2(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(input_tensor)
    x = Activation('relu')(x)
    x = Conv2D(filters1, kernel_size, padding='same', name=conv_name_base + '2a')(x)

    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters1, kernel_size, padding='same', name=conv_name_base + '2b')(x)

    x = layers.add([x, input_tensor])
    return x

## 設計一個先壓縮再回放的 ResNet V2 Block

In [ ]:
def Residual_block_v2(input_tensor, kernel_size, stage, block, compression_size=96, ouput_size=128):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(input_tensor)
    x = Activation('relu')(x)
    x = Conv2D(compression_size, kernel_size, padding='same', name=conv_name_base + '2a')(x)

    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(input_tensor)
    x = Activation('relu')(x)
    x = Conv2D(output_size, kernel_size, padding='same', name=conv_name_base + '2b')(x)

    x = layers.add([x, input_tensor])
    return x


- Inception A Block：請參考 image/inception-resnet-a.png
- Inception B Block：請參考 image/inception-resnet-b.png
- Inception C Block：請參考 image/inception-resnet-c.png

In [ ]:
def conv2d_bn(x, filters, kernel_size, padding='same', strides=(1, 1), batch_normalizer=True, activation='relu', name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3

    x = Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(x)

    if batch_normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)

    if activation:
        x = Activation(activation, name=act_name)(x)
    
    return x

In [ ]:
def residual_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

參考上方的 Residual_block()，搭建 Inception-ResNet 中的 Inception Block

In [ ]:
def inception_resnet_block(x, scale, block_type, activation='relu'):
    if block_type == 'incpetion_block_a':
        branch_0 = conv2d_bn(x, 32, (1, 1), name='inception_block_a_branch0_0')
        branch_1 = conv2d_bn(x, 32, (1, 1), name='inception_block_a_branch1_0')
        branch_1 = conv2d_bn(branch_1, 32, (3, 3), name='inception_block_a_branch1_1')
        branch_2 = conv2d_bn(x, 32, (1, 1), name='inception_block_a_branch_2_0')
        branch_2 = conv2d_bn(branch_2, 48, (3, 3), name='inception_block_a_branch_2_1')
        branch_2 = branch_2 = conv2d_bn(branch_2, 64, (3, 3), name='inception_block_a_branch_2_2')

        branches = [branch_0, branch_1, branch_2]
        mixed = Concatenate(axis=3)(branches)
        mixed = conv2d_bn(mixed, 384, (1, 1), name="inception_block_a_join_conv", activation=None)

    elif block_type == 'incpetion_block_b':
        branch_0 = conv2d_bn(x, 192, (1, 1), name='inception_block_b_branch0_0')
        branch_1 = conv2d_bn(x, 128, (1, 1), name='inception_block_b_branch1_0')
        branch_1 = conv2d_bn(branch_1, 160, (1, 7), name='inception_block_b_branch1_1')
        branch_1 = branch_1 = conv2d_bn(branch_1, 192, (7, 1), name='inception_block_b_branch1_1')

        branches = [branch_0, branch_1]
        mixed = Concatenate(axis=3)(branches)
        mixed = conv2d_bn(mixed, 1154, (1, 1), name="inception_block_a_join_conv", activation=None)

    elif block_type == 'incpetion_block_c':
        branch_0 = conv2d_bn(x, 192, (1, 1), name='inception_block_c_branch0_0')
        branch_1 = conv2d_bn(x, 192, (1, 1), name='inception_block_c_branch1_0')
        branch_1 = conv2d_bn(branch_1, 192, (1, 3), name='inception_block_c_branch1_1')
        branch_1 = branch_1 = conv2d_bn(branch_1, 192, (3, 1), name='inception_block_c_branch1_1')

        branches = [branch_0, branch_1]
        mixed = Concatenate(axis=3)(branches)

        mixed = conv2d_bn(mixed, 1792, (1, 1), name="inception_block_a_join_conv", activation=None)

    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "inception_block_a", "inception_block_b" or "inception_block_c", '
                         'but got: ' + str(block_type))
    
    
    # 確保輸入跟輸出深度相同
    check_depth = conv2d_bn(mixed, K.int_shape(x)[3], 1, activation=None)

    # 導入殘差結構
    # Lambda(function, output_shape, arguments)
    # function 的預設變數為 input，因此若要新增變數，就需要再 arguments 中自定一個變數
    # inputs[0] = x, inputs[1] = up
    x = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale, output_shape=K.int_shape(x)[1:], arguments={'scale': scale})([x, check_depth])
    
    if activation is not None:
        x = Activation(activation)(x)
    
    return x

In [ ]:
img_input = Input(shape=(224, 224, 32))

x = inception_resnet_block(img_input, 0.1, 'incpetion_block_a', activation='relu')
print(x)

## 測試

In [ ]:
img_input = Input(shape=(224, 224, 32))

x = inception_resnet_block(img_input, 0.1, 'incpetion_block_a', activation='relu')
print(x)

In [ ]:
img_input = Input(shape=(224,224,32))
x = inception_resnet_block(img_input, 0.1, 'incpetion_block_b', activation='relu')
print(x)

In [ ]:
img_input = Input(shape=(224,224,32))

x = inception_resnet_block(img_input, 0.1, 'incpetion_block_c', activation='relu')
print(x)

## 嘗試導入 Inception Block 到 vgg16_resnet_inception 中

In [ ]:
import import_ipynb
from Day019_Inception_HW import inception_v1_block, inception_v3_block

def vgg16_resnet_inception(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)
    
    x = conv2d_bn(img_input,64, (3, 3), activation='relu', padding='same', name='block1_conv1')
    x = conv2d_bn(x,64, (3, 3), activation='relu', padding='same', name='block1_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = conv2d_bn(x,128, (3, 3), activation='relu', padding='same', name='block2_conv1')
    x = conv2d_bn(x,128, (3, 3), activation='relu', padding='same', name='block2_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    # inception_block 可減少參數量
    x = inception_v1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
    x = inception_v1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_2')
    x = inception_v1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv1')
    x = conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv2')
    x = conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5 
    # inception_v3_block 放在較深的網路效果較好 
    x = inception_v3_block(x, ((128,), (192,256), (32,64), (64,)), 3, 'Block_4')
    x = inception_v3_block(x, ((128,), (192,256), (32,64), (64,)), 3, 'Block_5')
    x = inception_v3_block(x, ((128,), (192,256), (32,64), (64,)), 3, 'Block_6')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
       # 分類
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input

    # Create model.
    model = Model(inputs, x, name='vgg16')

   
    return model



In [ ]:
model = vgg16_resnet_inception(include_top=False)

In [ ]:
model.summary()